In [2]:
import os
import pandas as pd
from tqdm import tqdm 
from sentence_transformers import SentenceTransformer
import time
from lxml import etree
import pathlib
import re
from bs4 import BeautifulSoup

In [2]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

/home/lbartolome/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
def calculate_embedding(text_list):
    return [model.encode(text) for text in text_list]

def extract_heading_text(xml_content):
    try:
        # Parsear el contenido XML para crear un árbol de elementos
        root = etree.fromstring(xml_content)
        
        # Lista para guardar textos de las etiquetas 'heading'
        heading_texts = []        
        # Recorrer todos los elementos y extraer textos de las etiquetas 'heading'
        for element in root.iter('heading'):
            # Agregar el texto de la etiqueta 'heading' a la lista
            heading_texts.append(element.text.strip() if element.text else "")
        
        return heading_texts
    except etree.XMLSyntaxError:
        print("Error al parsear xml")
        return []  
    
def extract_all_xml_tags(xml_content):
    try:
        # Parsear el contenido XML para crear un árbol de elementos
        root = etree.fromstring(xml_content)
        # Inicializar una lista para guardar solo los nombres de las etiquetas
        all_tags = []
        # Recorrer todos los elementos en el árbol XML
        for element in root.iter():
            all_tags.append(element.tag)
    
        return all_tags
    
    except etree.XMLSyntaxError:
        print("Error al procesar")
        return []

In [7]:
from lxml import etree
def parse_xml_from_bytes(data_bytes):
    try:
        # Parsear el XML directamente desde bytes
        root = etree.fromstring(data_bytes)
        # Para extraer todo el texto concatenado de los elementos XML
        return ''.join(root.xpath('//text()'))
    except Exception as e:
        print(f"Error al parsear XML: {e}")
        return None

In [8]:
import re
import xml.etree.ElementTree as ET

def extract_text(xml_string):
    # Remove the initial double quotes and escaped newlines
    xml_string = xml_string.strip('"').replace('\\n', '')
    
    # Replace double double quotes with single double quotes
    xml_string = xml_string.replace('""', '"')
    
    # Use regex to fix XML declaration if it's malformed
    xml_string = re.sub(r'(<\?xml[^>]+>)', r'\1', xml_string)
    
    # Parse XML
    root = ET.fromstring(xml_string)
    
    # Extract text from all elements
    texts = []
    for elem in root.iter():
        if elem.text:
            texts.append(elem.text.strip())
    
    # Join all extracted texts with spaces
    extracted_text = ' '.join(texts)
    return extracted_text

In [6]:
df = pd.read_parquet("/export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated/procurements_file_10_containing_251208_docs_mark_fixed.parq")
df

,procurement_id,doc_name,content,alternative_lang,translated_content
0,ntp01156775,ntp01156775_Pliego_clausulas_administrativas_URI,b'<?xml version=\'1.0\' encoding=\'utf8\'?>\n<...,,
1,ntp01156775,ntp01156775_Anexos_pliegos_URI:1,"b""<?xml version='1.0' encoding='utf8'?>\n<docu...",,
2,ntp01156775,ntp01156775_Anexos_pliegos_URI:0,"b""<?xml version='1.0' encoding='utf8'?>\n<docu...",,
3,ntp01156775,ntp01156775_Anexos_pliegos_URI:3,"b""<?xml version='1.0' encoding='utf8'?>\n<docu...",,
4,ntp01156775,ntp01156775_Documento_Publicado_URI:3,"b""<?xml version='1.0' encoding='utf8'?>\n<docu...",,
...,...,...,...,...,...
251203,ntp01089479,ntp01089479_Documento_Publicado_URI:0,"b""<?xml version='1.0' encoding='utf8'?>\n<docu...",,
251204,ntp01089479,ntp01089479_Documento_Publicado_URI:2,b'<?xml version=\'1.0\' encoding=\'utf8\'?>\n<...,,
251205,ntp01089479,ntp01089479_Documento_Publicado_URI:5,"b""<?xml version='1.0' encoding='utf8'?>\n<docu...",,
251206,ntp01089479,ntp01089479_Documento_Publicado_URI:1,"b""<?xml version='1.0' encoding='utf8'?>\n<docu...",,


In [8]:
#df_[df_.procurement_id=="ntp00310764"].translated_content.iloc[0][0:100]

In [ ]:
#df_ = combined_df[combined_df.translated_content!=""]
#df_

In [ ]:
#df_ = combined_df[combined_df.translated_content!=""]
#extract_text(df_[df_.procurement_id=="ntp00310764"].translated_content.iloc[0])

In [ ]:
#parse_xml_from_bytes

In [3]:
df = pd.read_parquet("/export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated/procurements_file_2_containing_252000_docs_mark_fixed.parq")
df

,procurement_id,doc_name,content,alternative_lang,translated_content
0,ntp10244812,ntp10244812_Publicaciones_oficiales_URI,b'<?xml version=\'1.0\' encoding=\'utf8\'?>\n<...,,
1,ntp10244812,ntp10244812_ValidNoticeInfo_AdditionalPublicat...,b'<?xml version=\'1.0\' encoding=\'utf8\'?>\n<...,,
2,ntp01065327,ntp01065327_Pliego_clausulas_administrativas_URI,b'<?xml version=\'1.0\' encoding=\'utf8\'?>\n<...,,
3,ntp01065327,ntp01065327_Publicaciones_oficiales_URI:0,"b""<?xml version='1.0' encoding='utf8'?>\n<docu...",,
4,ntp01065327,ntp01065327_Pliego_Prescripciones_tecnicas_URI,"b""<?xml version='1.0' encoding='utf8'?>\n<docu...",,
...,...,...,...,...,...
251995,ntp00988283,ntp00988283_Pliego_Prescripciones_tecnicas_URI,"b""<?xml version='1.0' encoding='utf8'?>\n<docu...",,
251996,ntp00988283,ntp00988283_Anexos_pliegos_URI:2,"b""<?xml version='1.0' encoding='utf8'?>\n<docu...",,
251997,ntp00988283,ntp00988283_Anexos_pliegos_URI:1,b'<?xml version=\'1.0\' encoding=\'utf8\'?>\n<...,,
251998,ntp00988283,ntp00988283_Anexos_pliegos_URI:0,"b""<?xml version='1.0' encoding='utf8'?>\n<docu...",,


In [4]:
df[df.translated_content!=""]

,procurement_id,doc_name,content,alternative_lang,translated_content


In [9]:
# Ruta donde se encuentran los archivos Parquet originales a leer
path = '/export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated'
# Ruta donde se guardarán los archivos Parquet procesados
output_path = '/export/data_ml4ds/NextProcurement/PLACE/BSC_procesados_ca'

# Crear la lista de todos los archivos .parq en el directorio especificado
all_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.parq')]

palabras_clave = ['tecnicas']
# Expresión regular para identificar palabra clave
regex = '|'.join(palabras_clave)

# Usar tqdm en el bucle externo para ver el progreso en el procesamiento de archivos
for file in tqdm(all_files, desc="Procesando archivos"):
    # Leer el archivo Parquet
    df = pd.read_parquet(file)

    start_time = time.time()
    # Filtrado del df para filas donde la columna 'doc_name' contenga palabra clave
    df = df[df['doc_name'].str.contains(regex, na=False, case=False)]
    # Añadir una columna para identificar secciones del xml y meterlo en una columna llamada secciones.
    if "translated_content" in df.columns:
        #df['extracted_tags'] = df['translated_content'].apply(extract_all_xml_tags)
        #df['texto_heading'] = df['translated_content'].apply(extract_heading_text)
        
        df_cat = df[(df['translated_content'] != "") & (pd.notna(df['translated_content']))]
        indices_in_df_cat = df_cat.index
        df_no_cat = df[~df.index.isin(indices_in_df_cat)] # Filter df to get rows that are not in df_cat

        
        df_cat['extracted'] = df_cat['translated_content'].apply(extract_text)
        df_cat["translated"] = ["yes"] * len(df_cat)
        
        df_no_cat['extracted'] = df_no_cat['content'].apply(parse_xml_from_bytes)
        df_no_cat["translated"] = ["no"] * len(df_no_cat)
        df = pd.concat([df_cat,df_no_cat])
    else:
        #df['extracted_tags'] = df['content'].apply(extract_all_xml_tags)
        #df['texto_heading'] = df['content'].apply(extract_heading_text)
        df['extracted'] = df['content'].apply(parse_xml_from_bytes)
        df["translated"] = ["no"] * len(df)
        
    # Filtrar las filas donde la lista 'texto_heading' está vacía
    #df = df[df['texto_heading'].map(len) > 0]

    # Usar tqdm en el apply para ver el progreso del cálculo de embeddings
    #tqdm.pandas(desc="Calculando embeddings")
    #df['embeddings_heading'] = df['texto_heading'].progress_apply(calculate_embedding)
    print(df.columns)
    try:
        df = df[['procurement_id', 'doc_name', 'extracted', 'alternative_lang', 'translated']]#'embeddings_heading''extracted_tags', 'texto_heading', "translated_content"
    except Exception as e:
        print(e)
        df = df[['procurement_id', 'doc_name', 'extracted','translated']]#'embeddings_heading''extracted_tags', 'texto_heading', "translated_content"

    end_time = time.time()
    print(f"Archivo procesado: {file} en {end_time - start_time} segundos")
    
    # Construir el nombre del nuevo archivo conservando el nombre original pero cambiando la ruta
    new_file_path = os.path.join(output_path, os.path.basename(file))
    # Guardar el DataFrame en la nueva ruta
    df.to_parquet(new_file_path)

Procesando archivos:   0%|          | 0/17 [00:00<?, ?it/s]

Index(['procurement_id', 'doc_name', 'content', 'alternative_lang',
       'translated_content', 'extracted', 'translated'],
      dtype='object')
Archivo procesado: /export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated/procurements_file_15_containing_252064_docs_mark_fixed.parq en 33.9436149597168 segundos


Procesando archivos:   6%|▌         | 1/17 [01:15<20:04, 75.28s/it]

Index(['procurement_id', 'doc_name', 'content', 'alternative_lang',
       'translated_content', 'extracted', 'translated'],
      dtype='object')
Archivo procesado: /export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated/procurements_file_2_containing_252000_docs_mark_fixed.parq en 33.879493713378906 segundos


Procesando archivos:  12%|█▏        | 2/17 [03:09<24:31, 98.11s/it]

...

Procesando archivos:  88%|████████▊ | 15/17 [22:58<02:50, 85.28s/it]

Index(['procurement_id', 'doc_name', 'content', 'alternative_lang',
       'translated_content', 'extracted', 'translated'],
      dtype='object')
Archivo procesado: /export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated/procurements_file_3_containing_254088_docs_mark_fixed.parq en 30.490885257720947 segundos


Procesando archivos:  94%|█████████▍| 16/17 [24:21<01:24, 84.77s/it]/tmp/ipykernel_2804227/202573343.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['extracted'] = df_cat['translated_content'].apply(extract_text)
/tmp/ipykernel_2804227/202573343.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat["translated"] = ["yes"] * len(df_cat)
/tmp/ipykernel_2804227/202573343.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

Index(['procurement_id', 'doc_name', 'content', 'alternative_lang',
       'translated_content', 'extracted', 'translated'],
      dtype='object')
Archivo procesado: /export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated/procurements_file_16_containing_92602_docs_mark_fixed.parq en 11.316915512084961 segundos


Procesando archivos: 100%|██████████| 17/17 [24:50<00:00, 87.71s/it]
